In [2]:
!pip install tensorflow


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
from PIL import Image

In [39]:
# Get the absolute path to the dataset
dataset_path = "D:/Machine Learning/Image/Dataset/MGD/MGD2020/"
absolute_path = os.path.abspath(dataset_path)

In [40]:
# Load the pre-trained VGG16 model
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False


In [41]:
# Create a custom model for fine-tuning
model = tf.keras.Sequential([
    base_model,
    # reduce the complexity of the prev tensor derived from the input
    # by averaging the pixel distributions. So learning will be easy
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    # randomly deactivate nerouns to prevent overfitting on features by 
    # some neurons. 0.5 represent the % of neurons impacted by the dropout 
    #feature
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (gun or not)
])

In [42]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data preparation with data augmentation. Larger batch sizes are good to
# make the models to be more mature. However, it could lead to stuck due to
# memory shortages. Hence, trial and error expertimentations are required.

In [47]:
batch_size = 256
# augment the images on the go as per the specs given during the training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [48]:
train_generator = train_datagen.flow_from_directory(
    absolute_path,
    target_size=(224, 224),
    batch_size=batch_size,
    # augment the images considering correct and wrong representations.
    # this allow the model to learn with a comparative assesment.
    class_mode='binary',
    # can be either training or validation. In validation mode, adjust the
    # images to support the inferencing purposes.
    subset='training'
)

Found 6249 images belonging to 2 classes.


In [49]:
validation_generator = train_datagen.flow_from_directory(
    absolute_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 1562 images belonging to 2 classes.


In [ ]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
 8/25 [========>.....................] - ETA: 10:04:04 - loss: 0.2812 - accuracy: 0.9130